In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2

In [ ]:
image_size = (512, 512)

start_epoch = 1
end_epoch = 120

test_subset_id = 9

output_folder = "output/"
work_data_folder = "data/"
ground_truth_csv_path = "data/ground_truth.csv"
detection_pickle_path = "data/detection.pkl"

detections_output_path = "output/detections"

image_folder_path = "../data/images"

for folder in [output_folder, work_data_folder, detections_output_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
def read_detection_result():
    # if os.path.exists(detection_pickle_path):
    #     detection_dfs = pickle.load(open(detection_pickle_path, 'rb'))
    #     return detection_dfs
    detection_result_folder = "../output/valresults/caltech/h/off"

    detection_dfs = {}
    for epoch_id in tqdm(os.listdir(detection_result_folder), desc="test epoch"):
        current_folder_path = "{}/{}".format(detection_result_folder, epoch_id)

        detection_dfs.setdefault(epoch_id, {})
        for filename in os.listdir(current_folder_path):
            series_id = filename[0:-4]
            file = os.path.join(current_folder_path, filename)
            result_df = pd.read_csv(file, delimiter=" ", header=None)
            result_df.columns = ["z-index", "top_left_x", "top_left_y", "width", "height", "possibility"]
            detection_dfs.get(epoch_id).setdefault(series_id, result_df)

    pickle.dump(detection_dfs, open(detection_pickle_path, 'wb'))

    return detection_dfs

In [ ]:
test_detections = read_detection_result()
# test_detections["001"]

In [ ]:
def world_to_voxel_coord(world_coord, origin, spacing):
    stretched_voxel_coord = np.absolute(world_coord - origin)
    voxel_coord = stretched_voxel_coord / spacing
    return voxel_coord


def read_ground_truth_data():
    fid = open("../data/cache/test", 'rb')
    test_pickle = pickle.Unpickler(fid, encoding="latin1")
    test_cache = test_pickle.load()

    gt_df = pd.DataFrame(test_cache)
    gt_df["series_id"] = gt_df["filepath"].map(lambda x: x.split("/")[-2])
    gt_df["z-index"] = gt_df["filepath"].map(lambda x: int(x.split("/")[-1][2:-4]))

    return gt_df

In [ ]:
annotations = read_ground_truth_data()
# annotations.head()

In [ ]:
def compute_overlap_area_ratio(a, b):
    a_top_left_x, a_top_left_y, a_bottom_right_x, a_bottom_right_y = a
    b_top_left_x, b_top_left_y, b_bottom_right_x, b_bottom_right_y = b

    overlap_width = min(a_bottom_right_x, b_bottom_right_x) - max(a_top_left_x, b_top_left_x)
    if overlap_width <= 0:
        return 0

    overlap_height = min(a_bottom_right_y, b_bottom_right_y) - max(a_top_left_y, b_top_left_y)
    if overlap_height <= 0:
        return 0

    a_area = (a_bottom_right_x - a_top_left_x) * (a_bottom_right_y - a_top_left_y)
    b_area = (b_bottom_right_x - b_top_left_x) * (b_bottom_right_y - b_top_left_y)

    overlap_area = overlap_width * overlap_height
    total_area = a_area + b_area - overlap_area

    overlap_ratio = overlap_area / total_area
    return overlap_ratio

In [ ]:
miss_rate_x = []
miss_rate_y = []

number_of_test = len(annotations)

threshold = 0

for epoch_id in tqdm(test_detections.keys(), desc="epoch"):
    epoch_test_result = test_detections[epoch_id]

    hit_counter = 0

    for series_id in tqdm(epoch_test_result.keys(), desc="series id"):
        image_detections = epoch_test_result[series_id]
        image_detections["z-index"] = image_detections["z-index"].map(lambda x: round(x))
        image_detections["bottom_right_x"] = image_detections["top_left_x"] + image_detections["height"]
        image_detections["bottom_right_y"] = image_detections["top_left_y"] + image_detections["height"]
        image_detections["bbox"] =  list(image_detections[['top_left_x', 'top_left_y', "bottom_right_x", "bottom_right_y"]].to_records(index=False))

        nodules = annotations[annotations["series_id"] == series_id]

        for i, nodule in nodules.iterrows():
            z_index = nodule["z-index"]

            detections = image_detections[image_detections['z-index'] == z_index].reset_index()
            matched_detection = set()

            nodule_id = 0
            for bbox in nodule["bboxes"]:
                max_ratio = threshold
                best_detection_id = -1

                for detection_id, detection in detections.iterrows():
                    if detection_id in matched_detection:
                        continue

                    ratio = compute_overlap_area_ratio(bbox, detection["bbox"])

                    if ratio > max_ratio:
                        best_detection_id = detection_id
                        max_ratio = ratio

                if best_detection_id != -1:
                    hit_counter += 1
                    matched_detection.add(best_detection_id)
                    best_detection = detections.iloc[best_detection_id]

                    image = cv2.imread("{}/{}/z-{}.jpg".format(image_folder_path, series_id, z_index))

                    # add red detection rect
                    image = cv2.rectangle(image, (round(best_detection["top_left_x"]), round(best_detection["top_left_y"])), (round(best_detection["bottom_right_x"]), round(best_detection["bottom_right_y"])), (0, 0, 255), 1)

                    # add green annotation rect
                    image = cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 1)

                    folder = "{}/{}".format(detections_output_path, epoch_id)
                    if not os.path.exists(folder):
                        os.makedirs(folder)

                    cv2.imwrite("{}/{}-z{}-nodule{}-{}.jpg".format(folder, series_id, z_index, nodule_id, max_ratio), image)

            nodule_id += 1

    hit_rate = hit_counter / number_of_test
    miss_rate_y.append(1 - hit_rate)
    miss_rate_x.append(epoch_id)



 29%|██▉       | 17/59 [00:08<00:15,  2.75it/s]

In [ ]:
|plt.plot(miss_rate_x, miss_rate_y, label='MR')

In [ ]:
miss_rate_y